In [6]:
# Restart runtime after running this cell if needed
!pip install mediapipe==0.10.11 opencv-python --upgrade --quiet

#pip install numpy==1.26.4 mediapipe==0.10.11
# ========================================
# STEP 1: Upload an .mp4 video
# ========================================
from google.colab import files
uploaded = files.upload()  # Select a short .mp4 file with visible face

# ========================================
# STEP 2: Set up MediaPipe and OpenCV
# ========================================
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

input_video = list(uploaded.keys())[0]
cap = cv2.VideoCapture(input_video)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Initialize MediaPipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Drawing helper
mp_drawing = mp.solutions.drawing_utils

# Landmark indices for our expressions
LEFT_EYE = 159
RIGHT_EYE = 386
TOP_LIP = 13
BOTTOM_LIP = 14

# ========================================
# STEP 3: Define expression logic
# ========================================
def classify_expression(landmarks, w, h):
    left_eye_y = landmarks[LEFT_EYE].y * h
    right_eye_y = landmarks[RIGHT_EYE].y * h
    eye_diff = abs(left_eye_y - right_eye_y)

    top_lip_y = landmarks[TOP_LIP].y * h
    bottom_lip_y = landmarks[BOTTOM_LIP].y * h
    mouth_open = abs(bottom_lip_y - top_lip_y)

    if mouth_open > 25:
        return " Surprised"
    elif eye_diff < 2.5:
        return " Eyes Closed"
    elif mouth_open > 12:
        return " Smiling"
    else:
        return " Neutral"

# ========================================
# STEP 4: Process video frame by frame
# ========================================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            expression = classify_expression(face_landmarks.landmark, w, h)

            mp_drawing.draw_landmarks(
                frame,
                face_landmarks,
                mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None
            )
            cv2.putText(frame, expression, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0, 255, 0), 3)

    out.write(frame)

cap.release()
out.release()
print(" Video processing complete and saved to output.mp4")








'''

eye_closed = distance(landmarks[159], landmarks[145]) < threshold

# Check if mouth is wide open
mouth_open = distance(landmarks[13], landmarks[14]) > high_threshold

# Check if corners of mouth are pulled down (sad)
sad = (landmarks[61].y > landmarks[13].y) and (landmarks[291].y > landmarks[13].y)

# Check if eyebrows are pulled together (angry)
angry = (abs(landmarks[70].x - landmarks[336].x) < eyebrow_gap_thresh) and \
        (landmarks[70].y > landmarks[63].y)

# Check if one eyebrow is raised (skeptical)
skeptical = abs(landmarks[70].y - landmarks[336].y) > asymmetric_thresh

'''

Saving 7791920-hd_1920_1080_25fps.mp4 to 7791920-hd_1920_1080_25fps.mp4
 Video processing complete and saved to output.mp4


'\n\neye_closed = distance(landmarks[159], landmarks[145]) < threshold\n\n# Check if mouth is wide open\nmouth_open = distance(landmarks[13], landmarks[14]) > high_threshold\n\n# Check if corners of mouth are pulled down (sad)\nsad = (landmarks[61].y > landmarks[13].y) and (landmarks[291].y > landmarks[13].y)\n\n# Check if eyebrows are pulled together (angry)\nangry = (abs(landmarks[70].x - landmarks[336].x) < eyebrow_gap_thresh) and         (landmarks[70].y > landmarks[63].y)\n\n# Check if one eyebrow is raised (skeptical)\nskeptical = abs(landmarks[70].y - landmarks[336].y) > asymmetric_thresh\n\n'